In [8]:
import json
import os

# Ruta al archivo de configuración JSON
config_file_path = '../cfg-data/env-config.json'

def read_config(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
        return data

# Leer la configuración
config_data = read_config(config_file_path)

# Extraer los parámetros necesarios
INFLUXDB_IP = config_data['env']['INFLUXDB_IP']
INFLUXDB_DATABASE = config_data['env']['INFLUXDB_DATABASE']


In [1]:
%pip install pandas
%pip install influxdb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from influxdb_client import InfluxDBClient

# Configuración de InfluxDB
INFLUXDB_URL = 'http://localhost:8086'  # Cambia esto según tu configuración
INFLUXDB_TOKEN = 'my-secret-token'       # Cambia esto por tu token
INFLUXDB_ORG = 'SEAT S.A.'               # Cambia esto por tu organización
INFLUXDB_BUCKET = 'my-bucket'            # Cambia esto por tu bucket

# Consulta Flux
query = '''
from(bucket: "my-bucket")
  |> range(start: -1h)  // Consulta datos de la última hora
  |> limit(n: 1000)  // Limitar a los primeros 1000 resultados
'''

# Crear el cliente de InfluxDB
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)

# Ejecutar la consulta
query_api = client.query_api()
result = query_api.query(org=INFLUXDB_ORG, query=query)

# Convertir el resultado en un DataFrame de pandas
records = []
for table in result:
    for record in table.records:
        # Extraer solo las columnas que necesitas
        records.append({
            '_time': record.get_time(),
            '_field': record.get_field(),
            '_value': record.get_value()
        })

# Crear DataFrame con las columnas deseadas
df = pd.DataFrame(records)

# Convertir la columna '_time' a tipo datetime
df['_time'] = pd.to_datetime(df['_time'])

# Buscar filas duplicadas basadas en '_time' y '_field'
# y conservar solo la última entrada para cada combinación
df = df.drop_duplicates(subset=['_time', '_field'], keep='last')

# Pivotar el DataFrame para tener una columna por cada '_field'
df_pivot = df.pivot(index='_time', columns='_field', values='_value')

# Rellenar los valores NaN con el último valor válido
df_filled = df_pivot.fillna(method='ffill')

# Agrupar por el índice '_time' (sin truncar los segundos) y consolidar en una única fila por timestamp
df_grouped = df_filled.groupby(df_filled.index).first().reset_index()

# Mostrar el DataFrame agrupado
print(df_grouped)

# Guardar el DataFrame agrupado a un archivo CSV
df_grouped.to_csv('data/data_grouped.csv', index=False)

# Cerrar el cliente de InfluxDB
client.close()


/tmp/ipykernel_397523/3976245894.py:49: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df_pivot.fillna(method='ffill')
/tmp/ipykernel_397523/3976245894.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filled = df_pivot.fillna(method='ffill')


_field                            _time  \
0      2024-09-15 11:05:12.359354+00:00   
1      2024-09-15 11:05:12.359362+00:00   
2      2024-09-15 11:05:12.359363+00:00   
3      2024-09-15 11:05:12.359364+00:00   
4      2024-09-15 11:05:12.359365+00:00   
...                                 ...   
4845   2024-09-15 11:52:11.696776+00:00   
4846   2024-09-15 11:52:11.696777+00:00   
4847   2024-09-15 11:52:26.747203+00:00   
4848   2024-09-15 11:52:26.747212+00:00   
4849   2024-09-15 11:52:26.747213+00:00   

_field  120210R01_FB_SEAT_Statistik_ZAU.120210R01.N_NOK  \
0                                                   NaN   
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
...                                                 ...   
4845                                                0.0   
4846       